In [7]:
import os
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

#TODOSTILL
#use imputed data csv file

file_path = "dataset/imputed_combined_data.parquet"  # Adjust path if needed
df = pd.read_parquet(file_path)

# Display the first few rows
df.head()

max_length = df.groupby("patient_id").size().max()

def preprocess_data(output_file):

    data = df
    
    # Encode features (Gender: 0 = Female, 1 = Male)
    if 'Gender' in data.columns:
        data['Gender'] = LabelEncoder().fit_transform(data['Gender'].astype(str))
    
    feature_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    scaler = MinMaxScaler()
    data[feature_cols] = scaler.fit_transform(data[feature_cols])

    #ensures that al the patients have the same length

    padded_data = np.zeros((max_length, len(feature_cols)))
    sequence_length = min(max_length, data.shape[0])
    padded_data[:sequence_length, :] = data[feature_cols].iloc[:sequence_length].values

    # Save preprocessed data
    pd.DataFrame(padded_data, columns=feature_cols).to_csv(output_file, index=False)
    print(f"Preprocessed data saved to {output_file}")

# Example usage
output_file = "preprocessed_data.csv"
preprocess_data(output_file)

Preprocessed data saved to preprocessed_data.csv
